## CODIGO


# Model Design

In [13]:
# Diseño del modelo
import agentpy as ap
import numpy as np

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
import IPython

# Conexión mediante sockets 
import socket
import time

# Otras
import random

## Model Definition

In [14]:
# Estructuras de datos
paths_dictionary = {}
paths_array = []
parkingString = []

In [15]:
""" Agente vehículo"""
class VehicleAgent(ap.Agent):

  # Establecer configuración predeterminada
  def setup(self):
    self.map = self.model.map
    self.random = self.model.random
    self.steps = 0
    self.fuel = 0
    self.status = ''
    self.routeParking = None
    self.parking = None
    self.remainingParkingTime = None

  # Vehículo se estaciona 
  def park(self):
    self.status = 'parked'
    self.remainingParkingTime = random.randint(20, 40)

  # Vehículo se retira del estacionamiento
  def leaveParking(self):
    currentPos = self.map.positions[self]
    roadNeighbors = [i for i in self.map.neighbors(self, distance = 1) if i.type != 'ParkingAgent']
    posToMove = None

    for i in roadNeighbors:
      roadPos = self.map.positions[i]
      if abs(currentPos[0] - roadPos[0]) + abs(currentPos[1] - roadPos[1]) == 1:
        posToMove = roadPos

    if self.checkFront(posToMove):
      self.status = 'leaving'
      self.parking.availability = True
      self.parking = None
      self.routeParking = None
      self.remainingParkingTime = None
      return posToMove
    else:
      return None

  # Encontrar estacionamiento
  def findParking(self):
    pAgents = [i for i in self.model.map.agents if 'ParkingAgent' in i.type and i.availability]

    if len(pAgents) > 0:
      bestParking = None
      path = [None] * 99999

      for parkA in pAgents:

        greenFlag = True

        neighbors = self.map.neighbors(self, distance=0)
        roadNeighbors = [i for i in neighbors if ('VehicleAgent' not in i.type) and ('ParkingAgent' not in i.type)]
        currentPos = self.map.positions[roadNeighbors[0]]
        tempPath = []
        parkingPosition = self.map.positions[parkA]

        # Determinar la cantidad de movimientos para llegar a un estacionamiento
        while greenFlag and currentPos != parkingPosition:
          disToParking = abs(currentPos[0] - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
          
          # Salir de la ruta del camino al llegar al estacionamiento
          if disToParking == 1:
            nextPos = parkingPosition

          # Observar las carreteras de un solo sentido
          # Generar el movimiento hipotético para determinar la cantidad de pasos necesarios hacia el estacionamiento
          elif len(roadNeighbors) == 1:
            if(roadNeighbors[0].direction == 'North'):
              nextPos = (currentPos[0]-1, currentPos[1])
            elif(roadNeighbors[0].direction == 'South'):
              nextPos = (currentPos[0]+1, currentPos[1])
            elif(roadNeighbors[0].direction == 'West'):
              nextPos = (currentPos[0], currentPos[1]-1) 
            else:
              nextPos = (currentPos[0], currentPos[1]+1)

          # Revisa el mejor movimiento de un patrón hacia un estacionamiento en los cruces 
          elif len(roadNeighbors) > 1:
            directions = [i.direction for i in roadNeighbors]

            if 'South' in directions and 'West' in directions:
              southValue = abs((currentPos[0] + 2) - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
              westValue = abs(currentPos[0] - parkingPosition[0]) + abs((currentPos[1] - 2) - parkingPosition[1])
              if westValue < southValue:
                nextPos = (currentPos[0], currentPos[1]-1)
              else:
                nextPos = (currentPos[0]+1, currentPos[1])

            elif 'North' in directions and 'West' in directions:
              northValue = abs((currentPos[0] - 2) - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
              westValue = abs(currentPos[0] - parkingPosition[0]) + abs((currentPos[1] - 2) - parkingPosition[1])
              if northValue < westValue:
                nextPos = (currentPos[0]-1, currentPos[1])
              else:
                nextPos = (currentPos[0], currentPos[1]-1)

            elif 'South' in directions and 'East' in directions:
              southValue = abs((currentPos[0] + 2) - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
              eastValue = abs(currentPos[0] - parkingPosition[0]) + abs((currentPos[1] + 2) - parkingPosition[1])
              if southValue < eastValue:
                nextPos = (currentPos[0]+1, currentPos[1])
              else:
                nextPos = (currentPos[0], currentPos[1]+1)

            elif 'North' in directions and 'East' in directions:
              northValue = abs((currentPos[0] - 2) - parkingPosition[0]) + abs(currentPos[1] - parkingPosition[1])
              eastValue = abs(currentPos[0] - parkingPosition[0]) + abs((currentPos[1] + 2) - parkingPosition[1])
              if eastValue < northValue:
                nextPos = (currentPos[0], currentPos[1]+1)
              else:
                nextPos = (currentPos[0]-1, currentPos[1])

          # Determinar si la ubicación se encuentra en los bordes
          if nextPos[0] < 0 or nextPos[0] > self.map.shape[0]-1 or nextPos[1] < 0 or nextPos[1] > self.map.shape[1]-1 or len(tempPath) > 40 or len(tempPath) > len(path):
            greenFlag = False

          tempPath.append(nextPos)
          neighbors = self.map.neighbors(roadNeighbors[0], distance=1)
          roadNeighbors = [i for i in neighbors if ('VehicleAgent' not in i.type) and (self.map.positions[i] == nextPos)]
          currentPos = nextPos

        # Asignación del mejor patrón acorde a la cantidad de movimientos
        if greenFlag and len(tempPath) < len(path):
          path = tempPath
          bestParking = parkA
      
      if bestParking != None:
        # Reservar estacionamiento
        self.parking = bestParking
        self.routeParking = path
        self.status = 'found'
        bestParking.availability = False

    if self.parking:
      print("Id: " + str(self.id) + ", Vehículo: " + str(self.map.positions[self]) + ", ParkingCercano: " + str(self.map.positions[self.parking]))
      print(self.routeParking)
      print()
    else:
      print("Id: " + str(self.id) + ", Vehículo: " + str(self.map.positions[self]) + ", ParkingCercano: Ninguno")
      print()
  
  # Movimiento del vehículo 
  def movement(self):
    # Obtener los vecinos al vehiculo
    currentPosNeighbors = self.map.neighbors(self, distance = 0)
    currentRoads = [i for i in currentPosNeighbors if 'VehicleAgent' not in i.type and 'ParkingAgent' not in i.type]
    currentPos = self.map.positions[self]
    nextMovement = None

    # Movimiento del vehículo cuando está en busca de estacionamiento o cuando sólo está generando tráfico 
    # Elección de una dirección aleatoria en los  cruces.
    if self.status == 'leaving' or self.status == 'looking':
      # Si se llega al borde se remueve al agente del Grid
      if self.steps > 0 and (currentPos[0] == 0 or currentPos[0] == self.map.shape[0]-1 or currentPos[1] == 0 or currentPos[1] == self.map.shape[1]-1):
        self.map.remove_agents(self)
      else:
        decision = random.randint(0, len(currentRoads)-1)

        if(currentRoads[decision].direction == 'North'):
          nextMovement = (currentPos[0]-1, currentPos[1])

        elif(currentRoads[decision].direction == 'South'):
          nextMovement = (currentPos[0]+1, currentPos[1])

        elif(currentRoads[decision].direction == 'West'):
          nextMovement = (currentPos[0], currentPos[1]-1)
        
        else:
          nextMovement = (currentPos[0], currentPos[1]+1)
    
    # Ruta que sigue el auto hacia su estacionamiento reservado
    elif self.status == 'found':
      if self.routeParking:
        nextMovement = self.routeParking[0]
        #self.map.move_to(self, self.routeParking.pop(0))
      else:
        self.park()
    
    # Auto estacionado
    else:
      self.remainingParkingTime -= 1

      if self.remainingParkingTime <= 0:
        nextMovement = self.leaveParking()

    if nextMovement != None and self.checkFront(nextMovement):
      if self.status == 'found':
        self.routeParking.pop(0)
      self.map.move_to(self, nextMovement)
  
  # Evitar choques y coordinar los cruces
  def checkFront(self, nextMovement):
    frontNeighbors = [i for i in self.map.neighbors(self, distance = 1) if self.map.positions[i] == nextMovement]
    vehicleFront = [i for i in frontNeighbors if i.type == 'VehicleAgent']
    roadsFront = [i for i in frontNeighbors if 'VehicleAgent' not in i.type and 'ParkingAgent' not in i.type]
    
    # Si hay un carro adelante se regresa False
    if vehicleFront:
      return False
    # Si hay un cruce adelante
    if len(roadsFront) > 1:
      currentRoad = [i for i in self.map.neighbors(self, distance = 0) if 'VehicleAgent' not in i.type and 'ParkingAgent' not in i.type]
      currentDirection = currentRoad[0].direction
      crossDirections = [i.direction for i in roadsFront]
      crossPos = self.map.positions[roadsFront[0]]
      checkVehiclePos = None

      if 'South' in crossDirections and 'West' in crossDirections:
        if currentDirection == 'West':
          checkVehiclePos = (crossPos[0]-1, crossPos[0])
          
      elif 'North' in crossDirections and 'West' in crossDirections:
        if currentDirection == 'West':
          checkVehiclePos = (crossPos[0]+1, crossPos[0])
        
      elif 'South' in crossDirections and 'East' in crossDirections:
        if currentDirection == 'East':
          checkVehiclePos = (crossPos[0]-1, crossPos[0])

      elif 'North' in crossDirections and 'East' in crossDirections:
        if currentDirection == 'East':
          checkVehiclePos = (crossPos[0]+1, crossPos[0])
      
      # Retornar False si encuentra un auto que va a cruzar
      if checkVehiclePos != None:
        vehicleAboutToCross = [i for i in self.map.neighbors(self, distance = 1) if i.type == 'VehicleAgent' and self.map.positions[i] == checkVehiclePos]
        if vehicleAboutToCross:
          return False
    
    return True

In [16]:
""" Agente estacionamiento """
class ParkingAgent(ap.Agent):

  # Establecer configuración predeterminada
  def setup(self):
    self.map = self.model.map
    self.random = self.model.random
    self.id
    self.availability = True

  def setAvailability(self):
    # Condicional en dependencia a los agentes vehiculo
    pass

In [17]:
# Posiciones aleatorias
def randomPos(n):
    positions = [(0,4), (11,0), (15, 11), (4, 15)]
    return random.sample(positions, n)


In [18]:
""" Modelo de congestionamiento vehicular """
class CongestionModel(ap.Model):
  def setup(self):
    # Definir longitud de paths_array y vaciar diccionario
    paths_dictionary.clear()
    paths_array.clear()
    parkingString.clear()
    for _ in range(self.p.steps):
      paths_array.append("")

    # Creacion de Grid
    self.map = ap.Grid(self, (16, 16), track_empty = True)

    # Posiciones disponibles para estacionamientos
    parkingPositions = [(0,10), (0,12), (1,3), (1,5), (1,10), (1,12), (2,3), (2,5), (2,10), (2,12), (3,0), (3,1), (3,2), (3,6), (3,7), (3,8), (3,9), (3,13), (3,14), (5,0), (5,1), (5,2), (5,6), (5,7), (5,8), (5,9), (5,13), (5,14), (6,3), (6,5), (6,10), (6,12), (7,3), (7,5), (7,10), (7,12), (8,3), (8,5), (8,10), (8,12), (9,3), (9,5), (9,10), (9,12), (10,1), (10,2), (10,6), (10,7), (10,8), (10,9), (10,13), (10,14), (10,15), (12,1), (12,2), (12,6), (12,7), (12,8), (12,9), (12,13), (12,14), (12,15), (13,3), (13,5), (13,10), (13,12), (14,3), (14,5), (14,10), (14,12), (15,3), (15,5)]

    # Creacion de Agentes
    self.vehicleAgents = []

    parkingQuantity = self.p.parkingAgents
    if parkingQuantity > len(parkingPositions):
      parkingQuantity = len(parkingPositions)
    
    self.parkingAgents = ap.AgentList(self, self.p.parkingAgents, ParkingAgent)
    
    parkingSet = random.sample(parkingPositions, self.p.parkingAgents)

    pString = ""
    for a in parkingSet:
      pString += str(a[1]) + "," + str(abs(a[0] - 15)) + "/"

    parkingString.append(pString)

    # Generacion de Agentes Carretera y posiciones
    road = []
    pos = []

    for i in range(0,4,1):
      
      temp = ap.AgentList(self, 16)
      
      if (i == 0):
        temp.direction = "East"
        for j in range(0,16,1):
          pos += [(11,j)]
      elif (i == 1):
        temp.direction = "West"
        for j in range(0,16,1):
          pos += [(4,j)]
      elif (i == 2):
        temp.direction = "South"
        for j in range(0,16,1):
          pos += [(j,4)]
      else:
        temp.direction = "North"
        for j in range(0,16,1):
          pos += [(j,11)]
      
      road += temp
    
    self.roads = road
    
    # Añadir estacionamientos y calles al grid
    self.map.add_agents(self.parkingAgents, positions = parkingSet)
    self.map.add_agents(self.roads, positions = pos)
 
  def update(self):
    if self.t % self.p.vehicleSpawningWaitTime == 0 and self.t < self.p.steps:
      
      #self.record('Traffic', traffic())
      numVehicles = random.randint(1,4)
      positions = randomPos(numVehicles)

      if(numVehicles > 0):
        if numVehicles % 2 == 0:
          numLeaving = numVehicles//2
          numLooking = numVehicles//2    
        else:
          numLeaving = numVehicles//2
          numLooking = (numVehicles//2) + 1
        
        posLeaving = positions[:numVehicles//2]
        posLooking = positions[numVehicles//2:]

        vehiclesLeaving = ap.AgentList(self, numLeaving, VehicleAgent)
        vehiclesLooking = ap.AgentList(self, numLooking, VehicleAgent)

        vehiclesLeaving.status = 'leaving'
        vehiclesLooking.status = 'looking'

        self.vehicleAgents += vehiclesLeaving
        self.vehicleAgents += vehiclesLooking

        for i in range(0, len(vehiclesLeaving)):
          paths_dictionary[vehiclesLeaving[i].id] = [self.t, ""]

        for i in range(0, len(vehiclesLooking)):
          paths_dictionary[vehiclesLooking[i].id] = [self.t, ""]

        self.map.add_agents(vehiclesLeaving, positions = posLeaving)
        self.map.add_agents(vehiclesLooking, positions = posLooking)
        vehiclesLooking.findParking()

  def step(self):
    vehAgents = [i for i in self.map.agents if 'VehicleAgent' in i.type]

    for vehicle in vehAgents:
      if vehicle.status == 'looking':
        vehicle.findParking()

      #self.record(self.vehicleAgents.path)
      paths_dictionary[vehicle.id] = [paths_dictionary[vehicle.id][0], paths_dictionary[vehicle.id][1] + str(self.map.positions[vehicle][1]) + "," + str(abs(self.map.positions[vehicle][0]-15)) + "/"]

      vehicle.movement()
      vehicle.steps += 1      

  def end(self):
    pass
    #self.record(self.vehicleAgents.vars)

## Simulation Run

In [19]:
parameters = {
  #'vehicleAgents': 10,
  'parkingAgents': 14,
  'vehicleSpawningWaitTime': 5,
  'steps': 100,
  'seed': 42,
}

model = CongestionModel(parameters)
secondsPerStep = 1

In [20]:
""" Creación de una animación de la simulación """
def view_agents(Agent):
  color = 0

  if('VehicleAgent' in Agent.type):
    car = [i for i in Agent if i.type == 'VehicleAgent']

    if car[0].status == 'looking':
      color = 0
    elif car[0].status == 'found':
      color = 1
    elif car[0].status == 'parked':
      color = 2
    elif car[0].status == 'leaving':
      color = 3
  elif('ParkingAgent' in Agent.type):
    color = 4
  elif('Agent' in Agent.type):
    color = 5
  else:
    color = 6

  return color

def animation_plot(model, ax):
    attr_grid = model.map.apply(view_agents)
    # Color 0..3 -> cars, 4 -> parking, 5 -> road, 6 -> else
    color_dict = {0:'#C5BD28', 1:'#7FC97F', 2:'#3974B9', 3:'#C14242', 4:'#B8B8B8', 5:'#3B3B3B', 6:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)

fig, ax = plt.subplots()
model = CongestionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=1/secondsPerStep))

Id: 82, Vehículo: (0, 4), ParkingCercano: (1, 3)
[(1, 4), (1, 3)]

Id: 83, Vehículo: (15, 11), ParkingCercano: (14, 12)
[(14, 11), (14, 12)]

Id: 86, Vehículo: (11, 0), ParkingCercano: (12, 2)
[(11, 1), (11, 2), (12, 2)]

Id: 87, Vehículo: (15, 11), ParkingCercano: (13, 12)
[(14, 11), (13, 11), (13, 12)]

Id: 89, Vehículo: (0, 4), ParkingCercano: (2, 3)
[(1, 4), (2, 4), (2, 3)]

Id: 90, Vehículo: (4, 15), ParkingCercano: (5, 13)
[(4, 14), (4, 13), (5, 13)]

Id: 92, Vehículo: (11, 0), ParkingCercano: (9, 10)
[(11, 1), (11, 2), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10), (11, 11), (10, 11), (9, 11), (9, 10)]

Id: 94, Vehículo: (4, 15), ParkingCercano: (2, 10)
[(4, 14), (4, 13), (4, 12), (4, 11), (3, 11), (2, 11), (2, 10)]

Id: 97, Vehículo: (11, 0), ParkingCercano: (12, 13)
[(11, 1), (11, 2), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10), (11, 11), (11, 12), (11, 13), (12, 13)]

Id: 98, Vehículo: (15, 11), ParkingCercano: (8, 12)
[(1

In [21]:
# Establecer conexión con la parte gráfica
def set_paths_array():
    for key in paths_dictionary:
        paths_array[paths_dictionary[key][0]] += paths_dictionary[key][1] + "!"

set_paths_array()

# Definir puerto e IP para la conexión
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("127.0.0.1", 1101))
from_server = s.recv(4096)
print ("Received from server: ",from_server.decode("ascii"))

s.send(("timeStep:"+str(secondsPerStep)).encode())
time.sleep(2)

# Enviar coordenadas de los estacionaientos
s.send(("parkings:"+parkingString[0][:-1]).encode())
time.sleep(5)

# Enviar rutas de los vehículos
for i in range(0, len(paths_array)):
    s.send(("vehicles:"+paths_array[i]).encode())
    time.sleep(secondsPerStep)

s.close()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it